In [1]:
from utils.t5 import *
torch.cuda.empty_cache()
pl.seed_everything(42)

Global seed set to 42


42

In [2]:
torch.cuda.get_device_name(0)

'A100-SXM4-40GB'

In [7]:
input_data_name = "preprocessed.csv"                            
data_inpit_dir = "./Data/Preprocessed/"                              
output_dir = "./Data/Model/"
source_column = "outline"    
target_column =  "original_text"   
model_name_str = "fine_tunned_byt5_base"

no_workers = 1

imput_data_path = data_inpit_dir + input_data_name
new_model_name = "d-t5-{}".format(model_name_str)

In [8]:
data = pd.read_csv(imput_data_path)

data['target_text'] = data[target_column]
data['source_text'] = data[source_column]

data = data[["target_text", "source_text"]]

data['target_len'] = data["target_text"].str.split().str.len()
data['source_len'] = data["source_text"].str.split().str.len()

descriptives = data[['target_len','source_len']].describe()
descriptives

,target_len,source_len
count,281.000000,281.000000
mean,176.921708,155.252669
std,52.903898,53.038701
min,53.000000,41.000000
25%,140.000000,116.000000
50%,169.000000,150.000000
75%,211.000000,186.000000
max,332.000000,319.000000


In [9]:
train, dev_test = train_test_split(data, test_size = 0.3, random_state = 42)
dev, test = train_test_split(dev_test, test_size = 0.5, random_state = 42)

In [10]:
model = SimpleT5()

In [11]:
model.from_pretrained(model_type="t5",model_name = "t5-large") # large  "google/mt5-base" byt5-base

Downloading:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

In [12]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [13]:
model.train(train_df = train, 
            eval_df = dev, 
            source_max_token_len = 319,   
            target_max_token_len = 332,  
            batch_size = 1, max_epochs = 10, # 9 for base
            use_gpu = True,
            outputdir = output_dir,
            early_stopping_patience_epochs = 1,
            new_model_name = new_model_name) # 3

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-14fbf1d0-9297-ef80-4574-18d14d4b5767]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 1.2 B 
-----------------------------------------------------
1.2 B     Trainable params
0         Non-trainable params
1.2 B     Total params
4,912.734 Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/storage/brno3-cerit/home/vajp02/.conda/envs/myenv/lib/python3.9/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


RuntimeError: CUDA error: CUBLAS_STATUS_INTERNAL_ERROR when calling `cublasSgemmStridedBatched( handle, opa, opb, m, n, k, &alpha, a, lda, stridea, b, ldb, strideb, &beta, c, ldc, stridec, num_batches)`